In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import os, shutil

Using TensorFlow backend.


In [2]:
base_dir = 'datasets/dogs-vs-cats'

original_dir = os.path.join(base_dir, 'original')
original_train_dir = os.path.join(original_dir, 'train')

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

cats_train_dir = os.path.join(train_dir, 'cats')
cats_validation_dir = os.path.join(validation_dir, 'cats')

dogs_train_dir = os.path.join(train_dir, 'dogs')
dogs_validation_dir = os.path.join(validation_dir, 'dogs')

In [3]:
os.mkdir(train_dir)
os.mkdir(validation_dir)
os.mkdir(cats_train_dir)
os.mkdir(cats_validation_dir)
os.mkdir(dogs_train_dir)
os.mkdir(dogs_validation_dir)

In [4]:
def copy_images_to_folder(filename_pattern, start_range, stop_range, src_dir, dst_dir):
    filenames = [filename_pattern.format(i) for i in range(start_range, stop_range)]
    for filename in filenames:
        src = os.path.join(src_dir, filename)
        dst = os.path.join(dst_dir, filename)
        shutil.copyfile(src, dst)

copy_images_to_folder('cat.{}.jpg', 0, 10000, original_train_dir, cats_train_dir)
copy_images_to_folder('cat.{}.jpg', 10000, 12500, original_train_dir, cats_validation_dir)

copy_images_to_folder('dog.{}.jpg', 0, 10000, original_train_dir, dogs_train_dir)
copy_images_to_folder('dog.{}.jpg', 10000, 12500, original_train_dir, dogs_validation_dir)

In [3]:
batch_size = 20

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(50,50), 
                                                    batch_size=batch_size, 
                                                    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(validation_dir, 
                                                              target_size=(50,50), 
                                                              batch_size=batch_size, 
                                                              class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [4]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 128)         73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2, 2, 128)         147584    
__________

In [5]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit_generator(train_generator, 
                              steps_per_epoch=20000/batch_size, 
                              epochs=10, 
                              validation_data=validation_generator, 
                              validation_steps=5000/batch_size)

Epoch 1/10
1000/1000 [==============================] - 238s 238ms/step - loss: 0.6433 - acc: 0.6210 - val_loss: 0.6217 - val_acc: 0.6514
Epoch 2/10
1000/1000 [==============================] - 130s 130ms/step - loss: 0.5260 - acc: 0.7384 - val_loss: 0.4847 - val_acc: 0.7838
Epoch 3/10
1000/1000 [==============================] - 137s 137ms/step - loss: 0.4573 - acc: 0.7865 - val_loss: 0.4842 - val_acc: 0.7782
Epoch 4/10
1000/1000 [==============================] - 131s 131ms/step - loss: 0.4105 - acc: 0.8138 - val_loss: 0.4035 - val_acc: 0.8194
Epoch 5/10
1000/1000 [==============================] - 128s 128ms/step - loss: 0.3720 - acc: 0.8373 - val_loss: 0.4008 - val_acc: 0.8290
Epoch 6/10
1000/1000 [==============================] - 130s 130ms/step - loss: 0.3391 - acc: 0.8512 - val_loss: 0.3899 - val_acc: 0.8236
Epoch 7/10
1000/1000 [==============================] - 128s 128ms/step - loss: 0.3094 - acc: 0.8692 - val_loss: 0.3997 - val_acc: 0.8356
Epoch 8/10
1000/1000 [============

In [6]:
model.save('dogsvscats.h5')